In [ ]:
!pip uninstall tensorflow -y
!pip install tensorflow-gpu==2.1.0

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

#physical_devices = tf.config.list_physical_devices('GPU') 
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_dir = "/content/drive/My Drive/scdc2020/image11/train"
val_dir = "/content/drive/My Drive/scdc2020/image11/val"

In [ ]:
i = 0
X_train = np.array([])
y_train = np.array([])
for label in range(11):
    print(label)
    path = os.path.join(train_dir, str(label))
    files = os.listdir(path)
    for f in files:
        image = Image.open(path + "/" + f)  # uint8 image
        image = np.asarray(image).astype('uint8')
        image = np.expand_dims(image, axis=0)
        if i==0:
            X_train = image
            y_train = np.asarray(label)
            i += 1
        else:
            X_train = np.append(X_train, image, axis=0)
            y_train = np.append(y_train, np.asarray(label))

In [ ]:
i = 0
X_val = np.array([])
y_val = np.array([])
for label in range(11):
    print(label)
    path = os.path.join(val_dir, str(label))
    files = os.listdir(path)
    for f in files:
        image = Image.open(path + "/" + f)  # uint8 image
        image = np.asarray(image).astype('uint8')
        image = np.expand_dims(image, axis=0)
        if i==0:
            X_val = image
            y_val = np.asarray(label)
            i += 1
        else:
            X_val = np.append(X_val, image, axis=0)
            y_val = np.append(y_val, np.asarray(label))

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)

In [ ]:
p = 331

img_rows, img_cols, img_channel = p, p, 3

base_model = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

In [ ]:
add_model = tf.keras.Sequential()
add_model.add(tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(tf.keras.layers.Dropout(rate = 0.8))
#add_model.add(tf.keras.layers.Dense(units=256, activation=tf.nn.relu))
add_model.add(tf.keras.layers.Dense(units=11, activation=tf.nn.softmax))

model = tf.keras.Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=1e-6),
              metrics=['accuracy'])

In [ ]:
batch_size = 8
epochs = 1000

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255., dtype=np.float16)
train_datagen.fit(X_train)

import gc
gc.collect()

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255., dtype=np.float16)
val_datagen.fit(X_val)

gc.collect()

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    steps_per_epoch=X_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=val_datagen.flow(X_val, y_val, batch_size=batch_size),
    callbacks=[callback]
)

model.save("/content/drive/My Drive/scdc2020/image11/image11_model_1e_6.h5")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model auc')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()